In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import mean_squared_error

# Load and preprocess data
data = pd.read_csv("/content/database (1).csv")
data = data[['Latitude', 'Longitude', 'Depth', 'Magnitude']]

# Create a Cartopy map for visualization
fig = plt.figure(figsize=(10, 8))
ax = plt.axes(projection=ccrs.Robinson(central_longitude=0))

# Add map features
ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.BORDERS, linestyle=':')
ax.add_feature(cfeature.LAND, edgecolor='black')

# Plot earthquake data
sc = ax.scatter(data['Longitude'], data['Latitude'], c=data['Depth'], s=data['Magnitude'] * 5,
                cmap='viridis', transform=ccrs.PlateCarree())

# Add a colorbar
cbar = plt.colorbar(sc, orientation='vertical', label='Depth')

# Customize the colorbar
cbar.set_ticks([data['Depth'].min(), data['Depth'].max()])
cbar.set_ticklabels(['Shallow', 'Deep'])

# Set the title
plt.title('Earthquake Predictions')

# Split data into training and testing sets
X = data[['Latitude', 'Longitude', 'Depth']]
y = data[['Magnitude', 'Depth']]  # Choose either 'Magnitude' or 'Depth' as the target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build the neural network model
model = Sequential()
model.add(Dense(16, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(16, activation='relu'))
model.add(Dense(2, activation='linear'))  # Use 'linear' for regression

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mse'])

# Train the model
model.fit(X_train, y_train, batch_size=10, epochs=20, verbose=1, validation_data=(X_test, y_test))

# Make predictions
y_pred = model.predict(X_test)

# Calculate Mean Squared Error
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

# Show the Cartopy map
plt.show()
